In [1]:
%matplotlib inline

import numpy as np
# from numba.pycc import CC
from numba import njit, prange
import matplotlib.pyplot as plt

# cc = CC('my_numba_test')

In [2]:
# Test using numba parallel sum over rows
# @cc.export('do_numba_sum', 'f4(f4[:,:])')
@njit( parallel = True )
def do_numba_sum(vals):
    n = vals.shape[0]
    partials = np.zeros(n)
    for p in prange(n):
        partials[p] = np.sum(vals[p])
    return np.sum(partials)

In [3]:
# cc.compile()

In [4]:
n = 10000
# Generate some random numpy data for testing
data = np.arange(start=1, stop=(n**2)+1).reshape(n, n)
# print(data)

In [5]:
%%time 
# Test with basic numpy sum over rows
numpy_sum = 0

for row in np.arange(n):
    numpy_sum += np.sum(data[row])
    
print(numpy_sum)

max_val = n*n
calc_sum = (max_val * (max_val+1))//2
print(calc_sum)

5000000050000000
5000000050000000
CPU times: user 61.1 ms, sys: 9.72 ms, total: 70.9 ms
Wall time: 70.3 ms


In [6]:
%%time
x = np.arange(1).reshape(1,1)
print(x)
# do_numba_sum(x)
# print(numba_sum)
print(calc_sum)

[[0]]
5000000050000000
CPU times: user 87 µs, sys: 14 µs, total: 101 µs
Wall time: 102 µs


In [7]:
%%timeit -n 100000
numba_sum = do_numba_sum(data)
# print(numba_sum)
# print(calc_sum)

KeyboardInterrupt: 